# Home Depot Product Search Relevance
The challenge is to predict a relevance score for the provided combinations of search terms and products. To create the ground truth labels, Home Depot has crowdsourced the search/product pairs to multiple human raters.

## LabGraph Create
This notebook uses the LabGraph create machine learning iPython module. You need a personal licence to run this code.

In [36]:
import graphlab as gl
from nltk.stem import *

### Load data from CSV files

In [2]:
train = gl.SFrame.read_csv("../data/train.csv")

[INFO] This non-commercial license of GraphLab Create is assigned to thomasv1000@hotmail.fr and will expire on October 12, 2016. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-3252 - Server binary: C:\Users\T.Jaskula\AppData\Local\Continuum\Anaconda2\lib\site-packages\graphlab\unity_server.exe - Server log: C:\Users\T9773~1.JAS\AppData\Local\Temp\graphlab_server_1455037187.log.0
[INFO] GraphLab Server Version: 1.8.1


PROGRESS: Finished parsing file C:\Users\T.Jaskula\Python\HomeDepot\data\train.csv
PROGRESS: Parsing completed. Parsed 100 lines in 0.081008 secs.
------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[long,long,str,str,float]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------
PROGRESS: Finished parsing file C:\Users\T.Jaskula\Python\HomeDepot\data\train.csv
PROGRESS: Parsing completed. Parsed 74067 lines in 0.094009 secs.


In [3]:
test = gl.SFrame.read_csv("../data/test.csv")

PROGRESS: Finished parsing file C:\Users\T.Jaskula\Python\HomeDepot\data\test.csv
PROGRESS: Parsing completed. Parsed 100 lines in 0.451045 secs.
------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[long,long,str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------
PROGRESS: Finished parsing file C:\Users\T.Jaskula\Python\HomeDepot\data\test.csv
PROGRESS: Parsing completed. Parsed 166693 lines in 0.180018 secs.


In [4]:
desc = gl.SFrame.read_csv("../data/product_descriptions.csv")

PROGRESS: Finished parsing file C:\Users\T.Jaskula\Python\HomeDepot\data\product_descriptions.csv
PROGRESS: Parsing completed. Parsed 100 lines in 0.825083 secs.
------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[long,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------
PROGRESS: Read 61134 lines. Lines per second: 60763.3
PROGRESS: Finished parsing file C:\Users\T.Jaskula\Python\HomeDepot\data\product_descriptions.csv
PROGRESS: Parsing completed. Parsed 124428 lines in 1.24513 secs.


### Data merging

In [5]:
# merge train with description
train = train.join(desc, on = 'product_uid', how = 'left')

In [6]:
# merge test with description
test = test.join(desc, on = 'product_uid', how = 'left')

### Let's explore some data

Let's examine 3 different queries and products:
* first from the training set
* somewhere in the moddle in the training set
* the last one from the training set

In [7]:
first_doc = train[0]
first_doc

{'id': 2L,
 'product_description': 'Not only do angles make joints stronger, they also provide more consistent, straight corners. Simpson Strong-Tie offers a wide variety of angles in various sizes and thicknesses to handle light-duty jobs or projects where a structural connection is needed. Some can be bent (skewed) to match the project. For outdoor projects or those where moisture is present, use our ZMAX zinc-coated connectors, which provide extra resistance against corrosion (look for a "Z" at the end of the model number).Versatile connector for various 90 connections and home repair projectsStronger than angled nailing or screw fastening aloneHelp ensure joints are consistently straight and strongDimensions: 3 in. x 3 in. x 1-1/2 in.Made from 12-Gauge steelGalvanized for extra corrosion resistanceInstall with 10d common nails or #9 x 1-1/2 in. Strong-Drive SD screws',
 'product_title': 'Simpson Strong-Tie 12-Gauge Angle',
 'product_uid': 100001L,
 'relevance': 3.0,
 'search_term':

**'angle bracket'** search term is not contained in the body. **'angle'** would be after stemming however **'bracket'** is not.

In [8]:
middle_doc = train[37033]
middle_doc

{'id': 113228L,
 'product_description': 'PureBond Plywood Project Panels are a convenient and cost-effective way to build cabinets, furniture and other woodworking projects. It provides a beautiful wood veneer face bonded to a strong and flat wood core. These PureBond Project Panels are made with no added formaldehyde, eliminating the concern about off-gassing dangerous fumes during fabrication or when installed in your home. Their smaller size makes them easy to handle and allows you to order just the amount of wood you need. PureBond plywood, in Project Panels sizes or in full sheet sizes, are a Home Depot exclusive.California residents: see&nbsp;Proposition 65 informationDecorative mahogany veneer applied to both sides of this panelB-2 plain sliced mahogany - 7-ply constructionLight weight, all-wood veneer constructionPrecision-cut hardwood plywood panels in convenient small sizesCommon: 3/4 in. x 2 ft. x 4 ft.; Actual: 0.703 in. x 24 in. x 48 in.Grade: B-2',
 'product_title': '3/4 

only **'wood'** is present from search term

In [9]:
last_doc = train[-1]
last_doc

{'id': 221473L,
 'product_description': 'No. 918 Millennial Ryan heathered texture semi-sheer curtain is a casual solid that adds freshness and a finishing touch to any decor setting. Enhances privacy while allowing light to gently filter through. Clean, simple one-pocket pole top design can be used with a standard or decorative curtain rod. Mix and match with other solids and prints for a look that is all your own.Sheer panel, gently filters lightNo header pole top panelMachine washableWide array of colors to choose from100% polyesterContains 1-curtain panel',
 'product_title': 'LICHTENBERG Pool Blue No. 918 Millennial Ryan Heathered Texture Sheer Curtain Panel, 40 in. W x 63 in. L',
 'product_uid': 206650L,
 'relevance': 2.33,
 'search_term': 'fine sheer curtain 63 inches'}

**'sheer'** and **'courtain'** are present and that's all

### How many search terms are not present in description and title for ranked 3 documents
Ranked 3 documents are the most relevents searches, but how many search queries doesn't include the searched term in the description and the title

In [10]:
train['search_term_word_count'] = gl.text_analytics.count_words(train['search_term'])
ranked3doc = train[train['relevance'] == 3]
print ranked3doc.head()
len(ranked3doc)

+-----+-------------+-------------------------------+
|  id | product_uid |         product_title         |
+-----+-------------+-------------------------------+
|  2  |    100001   | Simpson Strong-Tie 12-Gaug... |
|  9  |    100002   | BEHR Premium Textured Deck... |
|  18 |    100006   | Whirlpool 1.9 cu. ft. Over... |
|  21 |    100006   | Whirlpool 1.9 cu. ft. Over... |
|  27 |    100009   | House of Fara 3/4 in. x 3 ... |
|  35 |    100011   | Toro Personal Pace Recycle... |
|  37 |    100011   | Toro Personal Pace Recycle... |
|  65 |    100016   | Sunjoy Calais 8 ft. x 5 ft... |
| 123 |    100023   | Quikrete 80 lb. Crack-Resi... |
| 162 |    100029   | DecoArt Americana Decor 16... |
+-----+-------------+-------------------------------+
+--------------------------------+-----------+-------------------------------+
|          search_term           | relevance |      product_description      |
+--------------------------------+-----------+-------------------------------+
|      

19125

In [11]:
words_search = gl.text_analytics.tokenize(ranked3doc['search_term'], to_lower = True)
words_description = gl.text_analytics.tokenize(ranked3doc['product_description'], to_lower = True)
words_title = gl.text_analytics.tokenize(ranked3doc['product_title'], to_lower = True)
wordsdiff_desc = []
wordsdiff_title = []
puid = []
search_term = []
ws_count = []
ws_count_used_desc = []
ws_count_used_title = []
for item in xrange(len(ranked3doc)):
    ws = words_search[item]
    pd = words_description[item]
    pt = words_title[item]
    diff = set(ws) - set(pd)
    if diff is None:
        diff = 0
    wordsdiff_desc.append(diff)
    
    diff2 = set(ws) - set(pt)
    if diff2 is None:
        diff2 = 0
    wordsdiff_title.append(diff2)
    
    puid.append(ranked3doc[item]['product_uid'])
    search_term.append(ranked3doc[item]['search_term'])
    ws_count.append(len(ws))
    ws_count_used_desc.append(len(ws) - len(diff))
    ws_count_used_title.append(len(ws) - len(diff2))
    
differences = gl.SFrame({"puid" : puid,
                         "search term": search_term,
                         "diff desc" : wordsdiff_desc,
                         "diff title" : wordsdiff_title,
                         "ws count" : ws_count, 
                         "ws count used desc" : ws_count_used_desc,
                         "ws count used title" : ws_count_used_title})

In [12]:
differences.sort(['ws count used desc', 'ws count used title'])

diff desc,diff title,puid,search term,ws count,ws count used desc,ws count used title
[concretetile],[concretetile],182917,concretetile,1,0,0
[confrence],[confrence],136503,confrence,1,0,0
"[over, berh, deck]","[over, berh, deck]",101795,berh deck over,3,0,0
"[barbque, grills]","[barbque, grills]",116142,barbque grills,2,0,0
[bearings],[bearings],149340,bearings,1,0,0
[medalion],[medalion],136409,medalion,1,0,0
"[wall, coatracks]","[wall, coatracks]",136407,wall coatracks,2,0,0
"[brite, scotch]","[brite, scotch]",149382,scotch brite,2,0,0
"[anti, microbial]","[anti, microbial]",149474,anti microbial,2,0,0
"[volt, 20, dcd790]","[volt, 20, dcd790]",116227,20 volt dcd790,3,0,0


In [13]:
print "No terms used in description : " + str(len(differences[differences['ws count used desc'] == 0]))
print "No terms used in title : " + str(len(differences[differences['ws count used title'] == 0]))
print "No terms used in description and title : " + str(len(differences[(differences['ws count used desc'] == 0) & 
                                                                        (differences['ws count used title'] == 0)]))

No terms used in description : 2666
No terms used in title : 2152
No terms used in description and title : 1206


In [14]:
import matplotlib.pyplot as plt
%matplotlib inline

### Stemming

In [69]:
#stemmer = SnowballStemmer("english")
stemmer = PorterStemmer()
def stem(word):
    singles = [stemmer.stem(plural) for plural in word.split()]
    text = ' '.join(singles)
    text = text.replace('\xe5', '')
    return text

In [70]:
print "Starting stemming train search term..."
stemmed = train['search_term'].apply(stem)
train['stem_search_term'] = stemmed

print "Starting stemming train product description..."
stemmed = train['product_description'].apply(stem)
train['stem_product_description'] = stemmed

print "Starting stemming train product title..."
stemmed = train['product_title'].apply(stem)
train['stem_product_title'] = stemmed

print "Starting stemming test search term..."
stemmed = test['search_term'].apply(stem)
test['stem_search_term'] = stemmed

print "Starting stemming test product description..."
stemmed = test['product_description'].apply(stem)
test['stem_product_description'] = stemmed

print "Starting stemming test product title..."
stemmed = test['product_title'].apply(stem)
test['stem_product_title'] = stemmed

UnicodeDecodeError: 'utf8' codec can't decode byte 0xe5 in position 0: unexpected end of data

### TF-IDF with linear regression

In [53]:
train['search_term_word_count'] = gl.text_analytics.count_words(train['stem_search_term'])
train_search_tfidf = gl.text_analytics.tf_idf(train['search_term_word_count'])

In [54]:
train['search_tfidf'] = train_search_tfidf

In [55]:
train['product_desc_word_count'] = gl.text_analytics.count_words(train['stem_product_description'])
train_desc_tfidf = gl.text_analytics.tf_idf(train['product_desc_word_count'])

In [56]:
train['desc_tfidf'] = train_desc_tfidf

In [66]:
train['product_title_word_count'] = gl.text_analytics.count_words(train['stem_product_title'])
train_title_tfidf = gl.text_analytics.tf_idf(train['product_title_word_count'])
train['title_tfidf'] = train_title_tfidf

RuntimeError: Runtime Exception. Traceback (most recent call last):
  File "<ipython-input-63-882ad86ba496>", line 8, in stem
  File "C:\Users\T.Jaskula\AppData\Local\Continuum\Anaconda2\lib\site-packages\nltk\stem\porter.py", line 632, in stem
    stem = self.stem_word(word.lower(), 0, len(word) - 1)
  File "C:\Users\T.Jaskula\AppData\Local\Continuum\Anaconda2\lib\site-packages\nltk\stem\porter.py", line 590, in stem_word
    word = self._step1ab(word)
  File "C:\Users\T.Jaskula\AppData\Local\Continuum\Anaconda2\lib\site-packages\nltk\stem\porter.py", line 288, in _step1ab
    if word.endswith("ied"):
UnicodeDecodeError: 'ascii' codec can't decode byte 0xe8 in position 2: ordinal not in range(128)


In [ ]:
train['distance_desc'] = train.apply(lambda x: gl.distances.cosine(x['search_tfidf'],x['desc_tfidf']))
train['distance_desc_sqrt'] = train['distance_desc'] ** 2
train['distance_title'] = train.apply(lambda x: gl.distances.cosine(x['search_tfidf'],x['title_tfidf']))
train['distance_title_sqrt'] = train['distance_title'] ** 3

In [ ]:
model1 = gl.linear_regression.create(train, target = 'relevance', 
                                         features = ['distance_desc', 'distance_desc_sqrt', 'distance_title', 'distance_title_sqrt'], 
                                         validation_set = None)

In [ ]:
#let's take a look at the weights before we plot
model1.get("coefficients")

In [ ]:
test['search_term_word_count'] = gl.text_analytics.count_words(test['stem_search_term'])
test_search_tfidf = gl.text_analytics.tf_idf(test['search_term_word_count'])
test['search_tfidf'] = test_search_tfidf
test['product_desc_word_count'] = gl.text_analytics.count_words(test['stem_product_description'])
test_desc_tfidf = gl.text_analytics.tf_idf(test['product_desc_word_count'])
test['desc_tfidf'] = test_desc_tfidf
test['product_title_word_count'] = gl.text_analytics.count_words(test['stem_product_title'])
test_title_tfidf = gl.text_analytics.tf_idf(test['product_title_word_count'])
test['title_tfidf'] = test_title_tfidf

test['distance_desc'] = test.apply(lambda x: gl.distances.cosine(x['search_tfidf'],x['desc_tfidf']))
test['distance_desc_sqrt'] = test['distance_desc'] ** 2
test['distance_title'] = test.apply(lambda x: gl.distances.cosine(x['search_tfidf'],x['title_tfidf']))
test['distance_title_sqrt'] = test['distance_title'] ** 3

In [ ]:
'''
predictions_test = model1.predict(test)
test_errors = predictions_test - test['relevance']
RSS_test = sum(test_errors * test_errors)
print RSS_test
'''

In [ ]:
predictions_test = model1.predict(test)
predictions_test

In [ ]:
submission = gl.SFrame(test['id'])

In [ ]:
submission.add_column(predictions_test)
submission.rename({'X1': 'id', 'X2':'relevance'})

In [ ]:
submission['relevance'] = submission.apply(lambda x: 3.0 if x['relevance'] > 3.0 else x['relevance'])
submission['relevance'] = submission.apply(lambda x: 1.0 if x['relevance'] < 1.0 else x['relevance'])

In [ ]:
submission['relevance'] = submission.apply(lambda x: str(x['relevance']))

In [ ]:
submission.export_csv('../data/submission2.csv', quote_level = 3)

In [ ]:
#gl.canvas.set_target('ipynb')